### Propuesta de trabajo (requerimientos de aprobación)
Transformaciones: Para este MVP no se te pide transformaciones de datos(aunque encuentres una motivo para hacerlo) pero trabajaremos en leer el dataset con el formato correcto. Puedes eliminar las columnas que no necesitan para responder las consultas o preparar los modelos de aprendizaje automático, y de esa manera optimizar el rendimiento de la API y el entrenamiento del modelo.

In [1]:
# Importar librería:
import pandas as pd
import numpy as np
import json
import ast
from textblob import TextBlob
from pandas import json_normalize
from sentiment_analysis_spanish import sentiment_analysis
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with open('output_steam_games.json', 'r') as f:
        steam_games_data = [json.loads(line) for line in f]

In [3]:
df_steam_games = pd.DataFrame(steam_games_data)
#df_steam_games.head(2)
df_steam_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items', 'items_count'],
      dtype='object')

In [40]:
df_steam_games.isnull().sum()

publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
user_id            32135
steam_id           32135
items              32135
items_count        32135
dtype: int64

In [4]:
# Eliminar filas con valores nulos de las columnas 'genres' y 'id':
df_steam_games = df_steam_games.dropna(subset=['genres','id'])

In [ ]:
# Eliminar columnas que no se usan
df_steam_games = df_steam_games.drop(['publisher','user_id', 'steam_id','items', 'items_count', 'app_name', 'url', 'reviews_url', 'discount_price','tags','developer'], axis=1)
df_steam_games.columns

Transformacion de Genres a columnas 

In [5]:
# Crear un nuevo DataFrame para almacenar los datos desglosados
#data_games_genres = df_steam_games.head(10000)

data_temp = pd.DataFrame()

for index, row in df_steam_games.iterrows():
    df_dummies = pd.get_dummies(row['genres'])
    df_dummies['id'] = row['id']
    df_dummies = df_dummies.groupby(by=["id"]).sum()
    data_temp = pd.concat([data_temp, df_dummies], ignore_index=False)

data_temp = data_temp.fillna(0)
data_temp

In [ ]:
#unimos las categorias al dataframe de games
merged_data_games = pd.merge(df_steam_games, data_temp, left_on='id', right_on='id')
merged_data_games.head(2)

Creamos la columna year a partir de release_date y cambiamos a numerico las columnas price y metascore

In [ ]:
#Convertimos la columna 'release_date' en datetime y creamos una nueva columna 'year'
merged_data_games['release_date'] = merged_data_games['release_date'].fillna('2023-01-01')
merged_data_games['release_date'] = pd.to_datetime(merged_data_games['release_date'], errors = 'coerce')
merged_data_games['year'] = merged_data_games['release_date'].dt.year
# Eliminar la columna 'release_date'
merged_data_games = merged_data_games.drop(columns=['release_date'])
merged_data_games.head(2)

In [ ]:
# Reemplazar 'Free to Play' con 0 y convertir la columna 'price' a tipo float
merged_data_games['price'] = pd.to_numeric(merged_data_games['price'], errors='coerce').fillna(0)

In [ ]:
# Reemplazar 'NA' y 'nan' con 0 en la columna 'metascore' y a tipo float
merged_data_games['metascore'] = merged_data_games['metascore'].replace(['NA'], 0)
merged_data_games['metascore'] = merged_data_games['metascore'].fillna(0).astype(float)
merged_data_games.head(2)

Creamos un nuevo Dataframe con los datos de los jugadores para recuperar los items

In [ ]:
data_players = pd.DataFrame(steam_games_data)
data_players.columns

In [ ]:
# Eliminar columnas que no se usan
data_players = data_players.drop(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer'], axis=1)
data_players.columns

In [ ]:
# Eliminar filas con valores nulos de la columna 'id':
data_players = data_players.dropna(subset=['user_id'])
data_players.head(2)

In [ ]:
#Se toma muestra de 10000 players para usarlo en la API posteriormente
data_games_players = data_players.head(10000)
data_games_players

Del dataframe de jugadores se extrae un nuevo dataframe con los items

In [ ]:
# Generamos un nuevo DF con los items de los jugadores
data_players_items = pd.DataFrame(columns=['user_id', 'steam_id', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks'])

for index, row in data_players.iterrows():
    temp_df = pd.DataFrame(row['items'])
    temp_df['user_id'] = row['user_id']
    temp_df['steam_id'] = row['steam_id']

    data_players_items = pd.concat([data_players_items, temp_df], ignore_index=True)

data_players_items.head()

In [ ]:
#Convertimos a float el campo 'playtime_forever'
data_players_items['playtime_forever'] = data_players_items['playtime_forever'].astype(float)

Preparamos los datos para incluir las horas jugadas de cada juego en el dataframe de juegos

In [ ]:
# Agrupamos por item_id
data_items = data_players_items.drop(['user_id', 'steam_id', 'playtime_2weeks'], axis=1)
data_items = data_items.groupby(by=['item_name']).sum()
data_items.info

In [ ]:
final_datagames = pd.merge(merged_data_games, data_items, left_on='title', right_on='item_name')
final_datagames.head(2)

In [ ]:
# Guardamos el dataframe de juegos con tiempo de juego como csv
final_datagames.to_csv('data_games.csv', header=True, index=False)

Preparamos un dataframe filtrado para agregar los generos y el año al dataframe de items

In [ ]:
data_games_filter = final_datagames.drop(['genres', 'specs', 'price', 'early_access', 'playtime_forever','id'], axis=1)
data_games_filter.columns

In [ ]:
merged_data_players = pd.merge(data_players_items, data_games_filter, left_on='item_name', right_on='title')
merged_data_players.head(2)

Limpiamos los nombres de las columnas de categorias en los dos dataframes

In [ ]:
merged_data_players = merged_data_players.rename(columns={'Animation &amp; Modeling':'Animation and Modeling',
                                                          'Design &amp; Illustration':'Design and Illustration'})
merged_data_players.columns

In [ ]:
final_datagames = final_datagames.rename(columns={'Animation &amp; Modeling':'Animation and Modeling',
                                                          'Design &amp; Illustration':'Design and Illustration'})
final_datagames.columns

Importamos el archivo de reviews

In [ ]:
with open('australian_user_reviews.json', 'r') as f:
        data = [ast.literal_eval(line) for line in f]
df = json_normalize(data)

In [ ]:
# Generamos un nuevo DF con los reviews de los jugadores
data_player_reviews = pd.DataFrame(columns=['funny','posted','last_edited','item_id','helpful','recommend','review'])

for index, row in df.iterrows():
    temp_df = pd.DataFrame(row['reviews'])
    temp_df['user_id'] = row['user_id']

    data_player_reviews = pd.concat([data_player_reviews, temp_df], ignore_index=True)

data_player_reviews.head()

In [ ]:
# convertimos la columna recommend a 0/1
data_player_reviews['recommend'] = data_player_reviews['recommend'].astype(int)
data_player_reviews.head(2)

In [ ]:
# Filtramos los reviews para anexarlo al dataframe de juegos
data_player_reviews_filter = data_player_reviews.drop(['funny', 'posted', 'last_edited', 'helpful'], axis=1)
recommended_games = data_player_reviews_filter.groupby(by=['item_id']).sum()
recommended_games['id'] = recommended_games.index
recommended_games['id'] = recommended_games['id'].astype(int)
recommended_games.head(2)

In [ ]:
# Obtenemos un nuevo dataframe con los juegos incluyendo la suma de recomendaciones 
data_games_wr = pd.merge(final_datagames, recommended_games, left_on='id', right_on='id')
data_games_wr.head(2)

In [ ]:
# Guardamos los CSV de final_datagames, merged_data_players y data_games_wr
final_datagames.to_csv('data_games.csv', header=True, index=False)
merged_data_players.to_csv('data_players.csv', header=True, index=False)
data_games_wr.to_csv('data_games_wr.csv', header=True, index=False)

### Desarrollo API: 
Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:
Debes crear las siguientes funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Código PlayTimeGenre

In [ ]:
def PlayTimeGenre(genre: str):
    if genre in data_games.columns:
        games_genre = data_games.loc[(data_games[genre]==1)]
        games_genre_year = games_genre.groupby(by=['year']).sum()
        year = games_genre_year[games_genre_year['playtime_forever'] == games_genre_year['playtime_forever'].max()].index[0]
        return year
    else:
        return 0

Código UserForGenre

In [ ]:
def UserForGenre(genre: str):
    if genre in data_games.columns:
        players_genre = data_players.loc[(data_players[genre]==1)]
        players_genre_group = players_genre.groupby(by=['user_id']).sum()
        max_player = players_genre_group[players_genre_group['playtime_forever'] == players_genre_group['playtime_forever'].max()].index[0]
        max_player_genre_items = data_players.loc[(data_players[genre]==1) & (data_players['user_id']==max_player)]
        max_player_genre_group = max_player_genre_items.groupby(by=['year']).sum()
        max_player_genre_group = max_player_genre_group[(max_player_genre_group['playtime_forever'] != 0)]
        playtime_max_player_genre = max_player_genre_group['playtime_forever'].to_dict()
        response = {"Usuario con más horas jugadas para Género " + genre : max_player, "Horas jugadas":[playtime_max_player_genre]}
        return response
    else:
        return 0

Código UsersRecommend

In [ ]:
def UsersRecommend(year: int):
    if year in data_games_wr['year'].values:
        data_games_wr_year = data_games_wr[data_games_wr['year'] == year]
        data_games_wr_year = data_games_wr_year.sort_values('recommend',ascending=False)
        recommended_games_list = data_games_wr_year['title'].head(3).tolist()
        response = [{"Puesto 1" : recommended_games_list[0]}, {"Puesto 2" : recommended_games_list[1]},{"Puesto 3" : recommended_games_list[2]}]
        return response
    else:
        return 0

Código UsersNotRecommend

In [ ]:
def UsersNotRecommend(year: int):
    if year in data_games_wr['year'].values:
        data_games_wr_year = data_games_wr[data_games_wr['year'] == year]
        data_games_wr_year = data_games_wr_year.sort_values('recommend',ascending=True)
        not_recommended_games_list = data_games_wr_year['title'].head(3).tolist()
        response = [{"Puesto 1" : not_recommended_games_list[0]}, {"Puesto 2" : not_recommended_games_list[1]},{"Puesto 3" : not_recommended_games_list[2]}]
        return response
    else:
        return 0

### Sentiments
Feature Engineering: 
En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}
Feature Engineering: En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [ ]:
with open('australian_user_reviews.json', 'r') as f:
    data = [ast.literal_eval(line) for line in f]
    #df = json_normalize(data)

In [3]:
data_reviews = pd.read_csv('data_reviews.csv')
data_reviews.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,1,Simple yet with great replayability. In my opi...,76561197970982479
1,NaN,"Posted July 15, 2011.",NaN,22200,No ratings yet,1,It's unique and worth a playthrough.,76561197970982479


In [5]:
# Función para asignar un valor numérico al sentimiento
def analyze_sentiment(reviews_list):
    sentiment_values = []
    for review_dict in reviews_list:
        review_text = review_dict  # Obtener el texto de la reseña
        if review_text:
            analysis = TextBlob(review_text)
            sentiment = analysis.sentiment.polarity
            if sentiment < 0:
                sentiment_values.append(0)  # Malo
            elif sentiment == 0:
                sentiment_values.append(1)  # Neutral
            else:
                sentiment_values.append(2)  # Positivo
    if sentiment_values:
        return max(set(sentiment_values), key=sentiment_values.count)
    else:
        return 1  # Valor predeterminado si no hay reseñas

In [ ]:
# Aplicar el análisis de sentimiento a cada lista de reseñas y crear la nueva columna
data_reviews['sentiment_analysis'] = data_reviews['review'].apply(analyze_sentiment)

In [ ]:
# Reemplazar la columna 'reviews' por 'sentiment_analysis'
data_reviews.drop(columns=['review'], inplace=True)
data_reviews.head(2)

In [ ]:
# Agregamos columna 'year' a partir de 'posted'
data_reviews['year'] = data_reviews['posted'].str[-4:]
data_reviews['year'] = pd.to_numeric(data_reviews['year'], errors='coerce').fillna(0)
data_reviews['year'].unique
data_reviews.head()

In [ ]:
# Guardamos el CSV de data_reviews
data_reviews.to_csv('data_reviews.csv', header=True, index=False)

Código sentiment_analysis

In [ ]:
def sentiment_analysis(year: int):
    if year in data_reviews['year'].values:
        if 0 in data_reviews['sentiment_analysis'].unique():
            neg = data_reviews['sentiment_analysis'].value_counts()[0]
        else:
            neg = 0
        if 1 in data_reviews['sentiment_analysis'].unique():
            neutr = data_reviews['sentiment_analysis'].value_counts()[1]
        else:
            neutr = 0
        if 2 in data_reviews['sentiment_analysis'].unique():
            pos = data_reviews['sentiment_analysis'].value_counts()[2]
        else:
            pos = 0
        response = {"Negative = " + neg, "Neutral = " + neutr, "Positive = " + pos}
        return response
    else:
        return 0

### Modelo de aprendizaje automático
Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. 
Para ello, te ofrecen dos propuestas de trabajo: 
En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. 

La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. Deben crear al menos uno de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:
Si es un sistema de recomendación item-item:
def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.
Si es un sistema de recomendación user-item:
def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [6]:
data_games = pd.read_csv('data_games_wr.csv')
data_games.head(2)

,genres,title,specs,price,early_access,id,metascore,Action,Casual,Indie,...,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting,year,playtime_forever,recommend
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,"['Single-player', 'Multi-player', 'Steam Tradi...",9.99,False,282010,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1997.0,2434.0,1
1,['Action'],Half-Life,"['Single-player', 'Multi-player', 'Valve Anti-...",9.99,False,70,96.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1998.0,474074.0,61


In [3]:
data_games.columns

Index(['genres', 'title', 'specs', 'price', 'early_access', 'id', 'metascore',
       'Action', 'Casual', 'Indie', 'Simulation', 'Strategy', 'Free to Play',
       'RPG', 'Sports', 'Adventure', 'Racing', 'Early Access',
       'Massively Multiplayer', 'Animation &amp; Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting', 'year', 'playtime_forever', 'recommend'],
      dtype='object')

In [7]:
# Eliminar la columna 'release_date'
ItemUser = data_games.drop(columns=['genres', 'title', 'specs', 'price', 'early_access', 'metascore', 'year', 'playtime_forever'])
ItemUser.head(2)

,id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting,recommend
0,282010,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,70,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61


In [11]:
ItemUser.columns

Index(['id', 'Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', 'Racing', 'Early Access',
       'Massively Multiplayer', 'Animation &amp; Modeling', 'Video Production',
       'Utilities', 'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting', 'recommend'],
      dtype='object')

In [12]:
# Buscar el cliente de posición 70:
item_similarity = pd.DataFrame(cosine_similarity(ItemUser))

In [13]:
# Cargar datos, asumiendo que ItemUser es tu DataFrame
# Calcular la similitud coseno entre los ítems
item_similarity = cosine_similarity(ItemUser)

In [15]:
# Crear un DataFrame para almacenar las puntuaciones de similitud
item_similarity_df = pd.DataFrame(item_similarity, index=ItemUser.index, columns=ItemUser.index)

In [ ]:
# Guardamos el CSV de data_reviews
item_similarity_df.to_csv('item_similarity_df.csv', header=True, index=False)

In [16]:
# Función para obtener recomendaciones para un ítem específico
def get_item_recommendations(item_id, top_n=5):
    # Obtener las puntuaciones de similitud para el ítem de entrada
    similar_items = item_similarity_df[item_id]
    # Ordenar los ítems según la similitud
    similar_items = similar_items.sort_values(ascending=False)
    # Obtener las recomendaciones (excluyendo el propio ítem)
    recommendations = similar_items.drop(item_id).head(top_n)
    return recommendations

In [20]:
# Ejemplo de recomendación para un ítem específico:
item_id = 70
recommendations = get_item_recommendations(item_id, top_n=5).index.to_list()
recommendations

[49, 64, 86, 2077, 2164]